In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal as Dec
import matplotlib.pyplot as plt
import sympy
import os
from datetime import datetime as dt
from datetime import timedelta as td
pd.options.mode.chained_assignment = None  # default='wsarn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# numerical results

In [100]:
folder_path = "numerical_detail_a_0_1"
num_results_df = pd.DataFrame()

for csv in sorted(os.listdir(folder_path)):
    row_dict = {}
    if "epoc" in csv:
        df = pd.read_csv(folder_path + "/" + csv)
        df = df.iloc[:-1]
        row_dict['pool'] = csv
        b = float(csv[:-9].replace("_", "."))
        row_dict['b'] = b
        row_dict['rebase_time'] = len(df) - 1
        row_dict['swap_time'] = df.iloc[-1]['epoc_end_index'] - 1
        row_dict['mean_stopping_time'] = row_dict['swap_time'] / row_dict['rebase_time']

        row_dict['pnl_mean'] = np.mean(df['end_wealth'] - df['invest_wealth'])
        row_dict['pnl_IL_mean'] = np.mean(df['end_wealth'] - df['holding_wealth'])
        row_dict['fee_wealth_mean'] = np.mean(df['end_wealth_fee'])
        row_dict['IL_mean'] = np.mean(df['end_wealth_in_pool'] - df['holding_wealth'])
        # row_dict['theo_IL'] = -(np.exp(b) - np.exp(-b)) / 4 * 20
        # row_dict['theo_pnl_mean'] = row_dict['theo_IL'] + row_dict['fee_wealth_mean']

        row_dict['pnl_apr'] = row_dict['pnl_mean'] / np.mean(df['invest_wealth'])
        row_dict['pnl_IL_apr'] = row_dict['pnl_IL_mean'] / np.mean(df['invest_wealth'])
        row_dict['fee_wealth_apr'] = row_dict['fee_wealth_mean'] / np.mean(df['invest_wealth'])
        row_dict['IL_mean_apr'] = row_dict['IL_mean'] / np.mean(df['invest_wealth'])
        # row_dict['theo_IL_apr'] = row_dict['theo_IL'] / np.mean(df['invest_wealth'])
        # row_dict['theo_pnl_apr'] = row_dict['theo_pnl_mean'] / np.mean(df['invest_wealth'])
        row_dict['effective_tx_ratio'] = sum(df['epoc_effective_tx_count']) / row_dict['swap_time']

        num_results_df = num_results_df.append(row_dict, ignore_index=True)

demo_df = num_results_df[['pool', 'b', 'rebase_time', 'swap_time', 'mean_stopping_time', 'fee_wealth_mean',
                          'IL_mean', 'pnl_mean', 'pnl_IL_mean', 'fee_wealth_apr', 'IL_mean_apr', 
                          'pnl_apr', 'pnl_IL_apr', 'effective_tx_ratio']]
demo_df
demo_df.to_csv("numerical_detail_a_0_1.csv")

# pool summary

In [131]:
source_folder = "pool_detail"
summary_folder = "pool_detail_summary"

for pool in sorted(os.listdir(source_folder)):
    if "." not in pool:
        num_results_df = pd.DataFrame()
        for csv in sorted(os.listdir(source_folder + "/" + pool)):
            row_dict = {}
            if "epoc" in csv:
                df = pd.read_csv(source_folder + "/" + pool + "/" + csv)
                df = df.iloc[:-1]
                row_dict['pool'] = pool
                if "optimum" not in csv:
                    b = float(csv[:-9].replace("_", "."))
                    row_dict['b'] = b
                else:
                    row_dict['b'] = "optimum"
                row_dict['rebase_time'] = len(df)
                row_dict['swap_time'] = df.iloc[-1]['epoc_end_index'] + 1
                row_dict['mean_stopping_index'] = row_dict['swap_time'] / row_dict['rebase_time']
                df['epoc_begin_timestamp'] = df['epoc_begin_timestamp'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))
                df['epoc_end_timestamp'] = df['epoc_end_timestamp'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))
                row_dict['start_time'] = df['epoc_begin_timestamp'].iloc[0]
                row_dict['end_time'] = df['epoc_end_timestamp'].iloc[-1]
                row_dict['mean_stopping_time'] = (row_dict['end_time'] - row_dict['start_time']) / row_dict['rebase_time']
                stopping_time = td(days=365) / row_dict['mean_stopping_time']
                row_dict['pnl_mean'] = np.mean(df['end_wealth'] - df['invest_wealth'])
                row_dict['pnl_IL_mean'] = np.mean(df['end_wealth'] - df['holding_wealth'])
                row_dict['fee_wealth_mean'] = np.mean(df['end_wealth_fee'])
                row_dict['IL_mean'] = np.mean(df['end_wealth_in_pool'] - df['holding_wealth'])
                # row_dict['theo_IL'] = -(np.exp(b) - np.exp(-b)) / 4 * 20
                # row_dict['theo_pnl_mean'] = row_dict['theo_IL'] + row_dict['fee_wealth_mean']

                row_dict['pnl_apr'] = row_dict['pnl_mean'] / np.mean(df['invest_wealth']) * stopping_time
                row_dict['pnl_IL_apr'] = row_dict['pnl_IL_mean'] / np.mean(df['invest_wealth']) * stopping_time
                row_dict['fee_wealth_apr'] = row_dict['fee_wealth_mean'] / np.mean(df['invest_wealth']) * stopping_time
                row_dict['IL_mean_apr'] = row_dict['IL_mean'] / np.mean(df['invest_wealth']) * stopping_time
                # row_dict['theo_IL_apr'] = row_dict['theo_IL'] / np.mean(df['invest_wealth'])
                # row_dict['theo_pnl_apr'] = row_dict['theo_pnl_mean'] / np.mean(df['invest_wealth'])
                row_dict['effective_tx_ratio'] = sum(df['epoc_effective_tx_count']) / row_dict['swap_time']

                num_results_df = num_results_df.append(row_dict, ignore_index=True)
            
            demo_df = num_results_df[['pool', 'b', 'rebase_time', 'swap_time', 'mean_stopping_index', 'start_time', 'end_time', 'mean_stopping_time',
                          'fee_wealth_mean', 'IL_mean', 'pnl_mean', 'pnl_IL_mean', 'fee_wealth_apr', 'IL_mean_apr', 
                          'pnl_apr', 'pnl_IL_apr', 'effective_tx_ratio']]

            demo_df.to_csv(summary_folder + "/" + pool + "_summary.csv")

# read summary

In [4]:
results_df = pd.DataFrame(columns=['pool', 'start_time', 'end_time', 'best_b', 'best_b_rebase_time', 'best_b_mean_stopping_index', 'best_b_mean_stopping_time', 'best_b_fee_wealth_mean', 'best_b_IL_mean', 'best_b_pnl_mean', 'bset_b_pnl_IL_mean', 'best_b_fee_wealth_apr', 'best_b_IL_mean_apr', 'best_b_pnl_apr', 'best_b_pnl_IL_apr', 'best_b_effective_tx_ratio',
                                                                    'optimum', 'optimum_rebase_time', 'optimum_mean_stopping_index', 'optimum_mean_stopping_time', 'optimum_fee_wealth_mean', 'optimum_IL_mean', 'optimum_pnl_mean', 'optimum_pnl_IL_mean', 'optimum_fee_wealth_apr', 'optimum_IL_mean_apr', 'optimum_pnl_apr', 'optimum_pnl_IL_apr', 'optimum_effective_tx_ratio'])
for csv in sorted(os.listdir("pool_detail_summary")):
    if ".csv" in csv:
        df = pd.read_csv(f"pool_detail_summary/{csv}")
        row = {}
        optimum = df.iloc[-1]
        try:
            best_b = df.sort_values("pnl_IL_apr", ascending=False).iloc[0]
            row['pool'] = best_b['pool']
            row['start_time'] = best_b['start_time']
            row['end_time'] = best_b['end_time']

            row['best_b'] = best_b['b']
            row['best_b_rebase_time'] = best_b['rebase_time']
            row['best_b_mean_stopping_index'] = best_b['mean_stopping_index']
            row['best_b_mean_stopping_time'] = best_b['mean_stopping_time']
            row['best_b_fee_wealth_mean'] = best_b['fee_wealth_mean']
            row['best_b_IL_mean'] = best_b['IL_mean']
            row['best_b_pnl_mean'] = best_b['pnl_mean']
            row['bset_b_pnl_IL_mean'] = best_b['pnl_IL_mean']
            row['best_b_fee_wealth_apr'] = best_b['fee_wealth_apr']
            row['best_b_IL_mean_apr'] = best_b['IL_mean_apr']
            row['best_b_pnl_apr'] = best_b['pnl_apr']
            row['best_b_pnl_IL_apr'] = best_b['pnl_IL_apr']
            row['best_b_effective_tx_ratio'] = best_b['effective_tx_ratio']

            row['optimum'] = optimum['b']
            row['optimum_rebase_time'] = optimum['rebase_time']
            row['optimum_mean_stopping_index'] = optimum['mean_stopping_index']
            row['optimum_mean_stopping_time'] = optimum['mean_stopping_time']
            row['optimum_fee_wealth_mean'] = optimum['fee_wealth_mean']
            row['optimum_IL_mean'] = optimum['IL_mean']
            row['optimum_pnl_mean'] = optimum['pnl_mean']
            row['optimum_pnl_IL_mean'] = optimum['pnl_IL_mean']
            row['optimum_fee_wealth_apr'] = optimum['fee_wealth_apr']
            row['optimum_IL_mean_apr'] = optimum['IL_mean_apr']
            row['optimum_pnl_apr'] = optimum['pnl_apr']
            row['optimum_pnl_IL_apr'] = optimum['pnl_IL_apr']
            row['optimum_effective_tx_ratio'] = optimum['effective_tx_ratio']

            results_df = results_df.append(row, ignore_index=True)
        
        except: pass

results_df.to_csv("pool_summary_results.csv")
print("done")

done


# get price data

In [116]:
# des_path = "pool_price_hist"
# for folder_name in sorted(os.listdir("pool_detail_backup")):
#     if "." not in folder_name:
#         df = pd.read_csv("pool_detail_backup/" + folder_name + "/0_2_transaction.csv")
#         df = df[['swap_index', 'current_price']]
#         df.to_csv(des_path + "/" + folder_name + "_price_hist.csv", index=False)
